# Introduction to Interactive Big Data Analysis with Spark

## Table of Content

1. [Curation]()
2. [Preparation]()
  1. [Data Importation](#2.A-Data-Importation)
  2. [Package Installation](#2.B-Package-Installation)
  3. [Package Importation](#2.C-Package-Importation)
  4. [Context Creation](#2.D-Context-Creation)
3. [Preprocessing]()
  1. [Creating an RDD](#3.A-Creating-an-RDD)
  2. [Getting Help](#3.B-Getting-Help)
  3. [Action on a Dataset](#4.-Action-on-a-Dataset)
  4. [Dataset Transformation](#5.-Dataset-Transformation)
  5. [Filtering a Dataset](#7.-Filtering-a-Dataset)
  6. [Caching a Dataset](#6.-Caching-a-Dataset)
4. [Processing]()
  1. []()
  2. []()
5. [Storage]()
6. [Recap](#6.-Recap)
7. [References](#7.-References)

## List of Exercises
1. [Exercise 1: How to create an RDD?](#Exercise-1)
1. [Exercise 1: How to Count?](#Exercise-1)
2. [Exercise 2: How to Transform?](#Exercise-2)
3. [Exercise 3: How to Filter?](#Exercise-3)
4. [Exercise 4: How to Sort?](#Exercise-4)

## 1. Curation

From Wikipedia:
> Data curation is a term used to indicate management activities related to organization and integration of data collected from various sources, annotation of the data, and publication and presentation of the data such that the value of the data is maintained over time, and the data remains available for reuse and preservation. Data curation includes "all the processes needed for principled and controlled data creation, maintenance, and management, together with the capacity to add value to data". In science, data curation may indicate the process of extraction of important information from scientific texts, such as research articles by experts, to be converted into an electronic format, such as an entry of a biological database. The term is also used in the humanities, where increasing cultural and scholarly data from digital humanities projects requires the expertise and analytical practices of data curation. In broad terms, curation means a range of activities and processes done to create, manage, maintain, and validate a component.

> According to the University of Illinois' Graduate School of Library and Information Science, "Data curation is the active and on-going management of data through its lifecycle of interest and usefulness to scholarship, science, and education; curation activities enable data discovery and retrieval, maintain quality, add value, and provide for re-use over time."

Curation being a field of its own, we will pass on the actual technics behind it. For this course, we will use a precurated dataset, the [eBooks@Adelaide dataset](https://ebooks.adelaide.edu.au/).

## 2. Preparation

### 2.A Data Importation
In order for all nodes of our cluster to access our data, we have previously imported the data in HDFS. Here are the commands that were used to import the data.

```Shell
hdfs dfs -mkdir /adelaide/
hdfs dfs -mkdir /adelaide/meta
hdfs dfs -mkdir /adelaide/page
hdfs dfs -put ~/datasets/meta/*.json /adelaide/meta
hdfs dfs -put ~/datasets/meta/*.json /adelaide/meta
```

We can confirm that the data is actually available by listing the content of the folders on HDFS.

In [ ]:
! hdfs dfs -ls /adelaide/page

In [ ]:
! hdfs dfs -ls /adelaide/meta

### 2.B Python Package Installation

To analyse our data, we will require some Python packages:  
- numpy for numeric data manipulation;
- networkx for network analysis;
- plotly for plotting;
- beautifulsoup4 to parse and extract data from HTML pages.

These packages have been installed with the following command:
```
pip install numpy networkx plotly beautifulsoup4
```

### 2.C Package Importation

In this notebook, we will use [Apache Spark](http://spark.apache.org) to analyze brievly the Adelaide University's Book Dataset.

First, we need to import Spark's Python module named `pyspark`. The module [`findspark`](https://github.com/minrk/findspark) is a wrapper that help us find the `pyspark`  module wherever it  is installed.

In [ ]:
import findspark
findspark.init()

import pyspark

We then import some Python standard modules that will help us during the analysis.

In [ ]:
import os
import json
import re

Finally, we import an interactive chart draing library [plotly](https://plot.ly/).

In [ ]:
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go

init_notebook_mode() # run at the start of every ipython notebook to use plotly.offline
                     # this injects the plotly.js source files into the notebook

### 2.D Context Creation

Once we have imported the required packages, we need to create a SparkContext. The context is an object that allows us to interact with the Spark cluster and create new resilient distributed dataset (RDD).

In [ ]:
conf = pyspark.SparkConf().setAppName("AdelaideNotebook")

try:
    sc = pyspark.SparkContext(conf=conf)
except:
    print("Warning : a SparkContext already exists.")

The context read the configuration file of Spark and automatically deduces the configuration of our cluster.

We can consult the dashboard of our Spark application.

## 3. Preprocessing
### 3.A Creating an RDD

We will now create an RDD. It represents the books' meta information in [JSON](http://www.json.org/).

In [ ]:
adelaide_meta_json = sc.textFile('hdfs://hdp:9000/adelaide/meta/*.json')

Here is an example of an entry of the `adelaide_meta_json` RDD:

```
{
  "@context": "http://schema.org", 
  "dateModified": "2014-02-26", 
  "image": "https://ebooks.adelaide.edu.au/b/bowen/marjorie/avenging-of-ann-leete/cover.jpg", 
  "author": "Bowen, Marjorie, 1885-1952", 
  "@type": "Book", 
  "source": "https://gutenberg.net.au/ebooks09/0900581.txt", 
  "inLanguage": "en", 
  "publisher": "The University of Adelaide Library", 
  "name": "The Avenging of Ann Leete", 
  "keywords": "Literature", 
  "url": "https://ebooks.adelaide.edu.au/b/bowen/marjorie/avenging-of-ann-leete/", 
  "description": "The Avenging of Ann Leete / Marjorie Bowen"
}
```

We can look at a few entries with the RDD's method `take` to get the first `K` elements of the meta information dataset. Here, `K = 4`.

In [ ]:
meta_first4 = adelaide_meta_json.take(4)
print(meta_first4)

Since `take` returns a list, we can iterate on the result and print it "prettily".

In [ ]:
for entry in meta_first4:
    print(entry)

#### Exercise 1 : Create an RDD

Create a new RDD named `adelaide_page_json` that contains the book URLs and its content as HTML code.

The path to the page files in HDFS is `/adelaide/page/`.

Here is an example of an entry of the `adelaide_page_json` RDD:
```
["https://ebooks.adelaide.edu.au/m/maupassant/guy/kiss/", "<!DOCTYPE html>\n\n<html> [...]"]
```

Then retrieve the first element of that RDD and show it on screen.

In [ ]:
#adelaide_page_json = <FILL_IN>
adelaide_page_json = sc.textFile('hdfs://hdp:9000/adelaide/page/*.json')

In [ ]:
#print(adelaide_page_json.take(1))

### 3.B Getting Help

At any moment, you can get help on a Python object using the `help()` function. For example, if we want to know more aboud the RDD's `take()` method.

In [ ]:
help(adelaide_meta_json.take)

### 3.C Action on a Dataset

The `take()` method is one among multiple available *actions* we apply on an RDD. An exhaustive list of action is available at the following URL:
https://spark.apache.org/docs/latest/programming-guide.html#actions

In case where we do not want to leave the notebook tab, we can call `help()` directly on an RDD.

In [ ]:
help(adelaide_meta_json)

Among the available actions, there is method named `count()`.

#### Exercise 2:  How to Count ?

Call the help function on the count method of the `adelaide_meta_json` to get to know more about the `count()` action. Then, apply this action on both RDDs and print the result to screen.

In [ ]:
# meta_count = adelaide_meta_json.count()
# page_count = adelaide_page_json.count()
# print(meta_count, page_count)

Each action apply on an RDD leads to the creation of one or many task and the production of a result. Every task executed in the same app can be visualised in the Spark's dashboard. In this interface, we can track the progress of a task, and check different performance measures on the task, for example its duration and cache statistics.

### 3.D Dataset Transformation

If we display the first 4 elements of our datasets that we retrieved earlier.

In [ ]:
meta_first4

We realize that the RDD is composed of by the lines of the input text files, but that is not possible to access to individual field in each dictionnary . **Why?**

In [ ]:
meta_first = adelaide_meta_json.first()
meta_first

The action `first()` as its name states, return the first entry of the dataset. We see that **each entry is a single string**. We will need to transform each entry of the RDD in order to convert the strings encoded in JSON in a Python dictionary. To do this, we will use the Python standard library function **`json.loads`** to convert each JSON encoded string into its Python equivalent.

First, lets test it `json.loads` on the previous first entry.

In [ ]:
json.loads(meta_first)

We now want to apply this transformation to every RDD's entry. The RDD's method `map(func)` returns a new distributed dataset formed by passing each element of the source through a function *func*.

In [ ]:
adelaide_meta = adelaide_meta_json.map(json.loads)

The evaluation of this transformation is *lazy*. Spark does not compute anything as long as a result is not requested by an action. To convince yourself, execute the preceding cell, then visit the Spark dashboard. You should see that no job have been added to the list.

To convince ourselves that the transformation will be successfully applied, we can retrieve the first element of the transformed RDD.

In [ ]:
print(adelaide_meta.first())

#### Exercise 3: How to Transform?

Apply the JSON transformation on the page RDD that we have created in exercise 1 and print the URL of the fifth element of that dataset.

In [ ]:
adelaide_page = adelaide_page_json.map(json.loads)
print(adelaide_page.take(5)[-1][0])

### 3.E Filtering a Dataset

#### 3.E.1 Filtering Bad Entries

Since we now have RDDs that are easier to manipulate, we can start the analysis. 

Our dataset was built by scraping the webpages of Adelaide University. However, during the process, some of the webpages could be fetched by our spider pogram. Therefore, in our dataset, we end up with two kinds of entry.

Good entry example:
```
{"@context": "http://schema.org", "dateModified": "2014-02-26", "image": "https://ebooks.adelaide.edu.au/b/bowen/marjorie/avenging-of-ann-leete/cover.jpg", "author": "Bowen, Marjorie, 1885-1952", "@type": "Book", "source": "https://gutenberg.net.au/ebooks09/0900581.txt", "inLanguage": "en", "publisher": "The University of Adelaide Library", "name": "The Avenging of Ann Leete", "keywords": "Literature", "url": "https://ebooks.adelaide.edu.au/b/bowen/marjorie/avenging-of-ann-leete/", "description": "The Avenging of Ann Leete / Marjorie Bowen"}
```

Bad entry example:
```
{"description": "ERROR_COMP_NOT_FOUND"}
```

For the next operation, we wish to only keep entries for which we at least know the name of the author and the title of the book. To do so, we first define a function that returns `True` if the fields `author` and `name` are in the dictionnary.

In [ ]:
def is_author_title_defined(record):
    return "author" and "name" in record

Try to answer the following quiz before executing the cell:  
* What sort of argument takes the `filter()` method?
* Is filter an action or a transformation?
* What does `filter()` return?

In [ ]:
adelaide_meta_filt = adelaide_meta.filter(is_author_title_defined)

#### Exercise 4

For the next exercise, you will design your own bad entry filter for the books' page RDD. The page RDD's entries are not dictonaries but lists. Here is an example of a bad entry:
```
["https://ebooks.adelaide.edu.au/d/dante/", "None"]
```

Write a function that will return `True` or `False` wether the entry is good or bad then create a new RDD named `adelaide_meta_filt` by applying your filter function of every entries of adelaide_page.

To assess your filter design, count the number of elements in the resulting RDD. How many entries have you filtered?

In [ ]:
adelaide_page_filt = adelaide_page.filter(lambda rec: rec[1] != "None")
adelaide_page_filt.count()

#### 3.E.2 Filtering Duplicate Entries

The meta-informations on each book have been recovered by scraping the website of [Adelaide University's eBook Libary](https://ebooks.adelaide.edu.au/). Since two pages could point to the same book, there is a possibility that a book is present more than once in our dataset.

#### Exercise 5

**To confirm that some books are present more than once in our dataset,  transform the dataset `adelaide_meta_filt` in a second that dataset that only includes URLs.**

In [ ]:
meta_urls = adelaide_meta_filt.map(lambda rec: rec['url'])# (<FILL IN>)

RDDs have a special method named `distinct()` that returns a new RDD containing strictly unique values. Next, we are going to call this method on our RDD of URLs and count the number of elements in that RDD.

In [ ]:
meta_urls.count() - meta_urls.distinct().count()

There is 305 duplicated entries in our dataset. To remove the duplicated entries, we will need to first associate an identifier that should be unique to each entry. We will refer to this id as a key. A unique identifier for a webpage is its URL. Since every book is associated to a URL, we will use this value as the key to our entries.

The Spark RDD method `keyBy()` allows to associate each entry in our dataset with a key. The key will be defined as an item of the record, in this case the url.

In [ ]:
adelaide_meta_url_key = adelaide_meta_filt.keyBy(lambda rec: rec['url'])

Each entry now has its own key, to convince ourselves, we can fetch the first element of that last RDD.

In [ ]:
adelaide_meta_url_key.first()

The last step is to keep only one value for each entry that shares the same key. To do this, we will apply a reduction operation, such that if we are given two records `a` and `b` with the same key, we only return the first record `a`. 

In [ ]:
adelaide_meta_unique = adelaide_meta_url_key.reduceByKey(lambda a, b: a)

This series of operations are called a reduction operation on a key-value pair RDD. We will get more into details in two sections.

#### Exercise 6

The page dataset may also includes duplicated pages. Identify what should be the key of that dataset then try to create a new dataset with only unique book entries. What is the structure of an entry after applying the `keyBy()` method? Do we need to transform this dataset or could it be reduced directly?

Count the number of elements in the resulting RDD to confirm your transformation.

In [ ]:
adelaide_page_unique = adelaide_page_filt.reduceByKey(lambda a, b: a)
print(adelaide_page_unique.count())

### 3.F Caching a Dataset

When we expect to operate frequently on the same dataset, it can be useful to tell Spark to keep it in memory.

To do so, we use the `cache()` method.

In [ ]:
adelaide_meta_unique.cache()

The RDDs stored in memory are displayed in the **Storage** section of Spark web interface. Note that datasets are not loaded in memory until an action is called on them. 

Action on cached dataset are much faster than non cached dataset. But in order to be cached, an action must first be applied on the dataset. Based on that, try to explain the execution time for the following cells.

In [ ]:
%time adelaide_meta_unique.count()

In [ ]:
% time adelaide_meta_unique.count()

#### Exercise 7

Cache the RDD from exercise 6 and evaluate how long does it takes to retrieve the first 5 elements before and after caching. What happens when there is not enough memory to cache an RDD? Can you figure how to *uncache* an RDD?

In [ ]:
adelaide_page_unique.cache()

In [ ]:
% time first_5pages = adelaide_page_unique.take(5)

In [ ]:
% time first_5pages = adelaide_page_unique.take(5)

## 4. Processing

To grasp the extent of our dataset, we can first count to number of entries it contains

In [ ]:
adelaide_meta.count()

Each entry is of our dataset is a dictionary. Each dictionary can contain a variable number of keys and each dictionary in our dataset do not necessarily shared the same keys.

We can extract the keys from each dictionary and count how many times they are present. To access, the key of a dictionary, we can use the method `keys()`.

In [ ]:
adelaide_meta.first().keys()

We want to apply this method to every dictionary in our dataset, so that would be a map. However, if we simply apply a map, we will get an RDD of key-lists. What we truly want is to merge the list to get an RDD of keys. 

Spark has a function to merge the iterable return by a function, the `flatMap`.

In [ ]:
adelaide_keys = adelaide_meta.flatMap(lambda rec: rec.keys())

We can inspect the first 5 elements of our dataset.

In [ ]:
adelaide_keys.take(5)

We are now interested in finding the frequency of each key. This will give us an idea of the completeness of our dataset. To compute the key frequency, we will need to apply a classic map-reduce pattern.

First, we need to pair each key with the basic frequency value 1. This is the map operation.

In [ ]:
adelaide_key_value = adelaide_keys.map(lambda key: (key, 1))

Again, we can inspect the result of the transformation by looking at the first element

In [ ]:
adelaide_key_value.first()

What we will do next is add the value associated with each pair that shares the same key. This is the reduce operation.

In [ ]:
from operator import add

adelaide_agg = adelaide_key_value.reduceByKey(add)

Finally, we can collect our transformed RDD. Key-Value pair RDDs have a special method `collectAsMap` that returns the result as a dictionnary.

In [ ]:
adelaide_agg.collectAsMap()

We observe that only a few keys are available in most dictionaries of our datasets. We should therefore restrict our analysis to these fields or create new fields from the frequent one.

### 4.A Valorizing data by transforming the dataset

In [ ]:
def process_name_birth_death(record):
    author = record.get('author', None)
    if author:
        author = author.strip()
        # Remove trailing dot
        if '.' == author[-1]:
            author = author[:-1]
        try:
            lastname, firstname, birth_death = author.split(',')
        except ValueError:
            return record
        try:
            birth, death = re.findall('\d+', birth_death)
        except ValueError:
            return record
        record['author_lastname'] = lastname.strip()
        record['author_firstname'] = firstname.strip()
        record['author_birth'] = int(birth)
        record['author_death'] = int(death)
    return record

In [ ]:
adelaide_meta_val = adelaide_meta_unique.mapValues(process_name_birth_death).values()

In [ ]:
adelaide_meta_val.take(2)

In [ ]:
def convert_dateCreated(record):
    if 'dateCreated' in record:
        dates = re.findall('\d+', record['dateCreated'])
        if len(dates) > 0:
            date = int(dates[0])
            # Check if the date is before common era
            if re.match(r'BC|bc|BCE|bce', record['dateCreated']):
                date *= -1
            record['dateCreated'] = date
        else:
            del record['dateCreated']
    return record

In [ ]:
adelaide_meta_val = adelaide_meta_val.map(convert_dateCreated)

### 4.B First analysis: authors' life expectancy

In [ ]:
def compute_age(rec):
    """Compute the age of an author when it died
    based on its year of birth and death.
    """
    if 'author_birth' and 'author_death' in rec:
        birth, death =  rec['author_birth'], rec['author_death']
        if birth < death:
            return death - birth
        else:
            # If year of birth is greater than year of death the
            # author was born in BCE. Do you think this is correct
            # in every cases?
            return birth - death
    else:
        return None

age_frequency = adelaide_meta_val.map(compute_age)\
                                 .countByValue()

Visualization can be done with multiple tools, here we use plotly.

In [ ]:
data = [
    go.Bar(
        x=list(age_frequency.keys()),
        y=list(age_frequency.values()),
    )
]

layout = go.Layout(
    title="Adelaide authors life expectancy",
    xaxis=dict(
        title='life expectancy (years)',
    ),
    yaxis=dict(
        title='number of authors',
    ),    
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

A surprising number of authors died at the age of 43. There is either a pattern with authors, or we have commited a mystake in our analysis.

What happens if an author has written more than one book? We need to remember that our dataset is composed of books, not authors. If we want to produce statistics on the authors, we need to keep only distinct authors.

### 4.C Second analysis: authors' life expectancy... done correctly

In [ ]:
def retrieve_name_age(rec):
    age = compute_age(rec)
    lastname = rec['author_lastname']
    firstname = rec['author_firstname']
    return firstname, lastname, age

authors = adelaide_meta_val.filter(lambda rec: 'author_lastname' in rec)\
                           .map(retrieve_name_age)

In [ ]:
unique_authors = authors.distinct()
age_frequency2 = unique_authors.map(lambda tup: tup[2]).countByValue()

In [ ]:
data = [
    go.Bar(
        x=list(age_frequency2.keys()),
        y=list(age_frequency2.values()),
    )
]

layout = go.Layout(
    title="Adelaide authors life expectancy",
    xaxis=dict(
        title='life expectancy (years)',
    ),
    yaxis=dict(
        title='number of authors',
    ),    
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

## 6. Storage



## 7. Recap
### Preprocessing the pages to extract the text

In [ ]:
from itertools import chain
from operator import itemgetter
from bs4 import BeautifulSoup

In [ ]:
url, book1 = adelaide_page.first()

In [ ]:
adelaide_page_unique = adelaide_page.groupByKey()\
                                    .mapValues(list)\
                                    .mapValues(itemgetter(0))

In [ ]:
def extract_text(page):
    if page:
        soup = BeautifulSoup(page, 'html.parser')
        it = chain(soup.findAll(['meta', 'script', 'head']),
                   soup.findAll('div', {"id" : "controls"}),
                   soup.findAll('div', {"class" : "contents"}),
                   soup.findAll('div', {"class" : "titleverso"}),
                   soup.findAll('div', {"class" : "colophon"}),
                   soup.findAll('span', {"class" : "author"}))
        for div in it:
            div.extract()
        return soup.get_text().strip()

In [ ]:
adelaide_text = adelaide_page_unique.mapValues(extract_text)

In [ ]:
adelaide_text.count()

### Processing: Analysing the work of an era

We have an dataset of 4371 different books written at different times. Lets suppose we want to study the texts of the books written during the 1901–1939 Modernism era.

First we need to identify which books in our dataset were written during this era.

In [ ]:
modern_era_books = adelaide_meta_val.filter(lambda rec: 1900 < rec.get('dateCreated', 0) < 1938)

Some of these books are not necessarily in English. We therefore need to apply a second filter on the language (`inLanguage`).

In [ ]:
modern_era_en_books = modern_era_books.filter(lambda rec: rec.get('inLanguage', '') == 'en')

We can now count how many English books from our dataset were written during this era.

In [ ]:
modern_era_en_books.count()

We can also count the number of distinct authors:

In [ ]:
modern_era_books.map(lambda rec: rec['author']).distinct().count()

The meta information about each book and the book's text are stored in two separate RDDs. In order to retrieve the texts written during the modernism era, we will need to join the RDD of modern book era metainformation and the RDD of books' text.

To do so, we will first need to define the modern era book RDD as an RDD of key-value pairs. 

In [ ]:
modern_era_books_kv = modern_era_books.keyBy(lambda rec: rec['url'])

In [ ]:
modern_era_books_kv.keys().distinct().count()

Then, we can join the RDD of meta information on modern era books with the RDD of books' text to access the 

In [ ]:
modernism_meta_text = modern_era_books_kv.join(adelaide_text)

Finally, we join the words of each text.

In [ ]:
import string

In [ ]:
modernism_text = modernism_meta_text.mapValues(lambda x: x[1])
modernism_word = modernism_text.flatMapValues(string.split)

In [ ]:
from string import punctuation
def remove_punctuations(word):
    return re.sub(r'[{}‘—’”“]'.format(punctuation), " ", word).strip()

stopwords  = set(['all', 'pointing', 'four', 'go', 'oldest', 'seemed', 'whose', 'certainly',
'young',  'presents', 'to', 'asking', 'those', 'under', 'far', 'every',
'presented', 'did',  'turns', 'large', 'p', 'small', 'parted', 'smaller',
'says', 'second', 'further',  'even', 'what', 'anywhere', 'above', 'new',
'ever', 'full', 'men', 'here', 'youngest',  'let', 'groups', 'others', 'alone',
'along', 'great', 'k', 'put', 'everybody', 'use',  'from', 'working', 'two',
'next', 'almost', 'therefore', 'taken', 'until', 'today',  'more', 'knows',
'clearly', 'becomes', 'it', 'downing', 'everywhere', 'known', 'cases',  'must',
'me', 'states', 'room', 'f', 'this', 'work', 'itself', 'can', 'mr', 'making',
'my', 'numbers', 'give', 'high', 'something', 'want', 'needs', 'end', 'turn',
'rather', 'how', 'y', 'may', 'after', 'such', 'man', 'a', 'q', 'so', 'keeps',
'order', 'furthering',  'over', 'years', 'ended', 'through', 'still', 'its',
'before', 'group', 'somewhere',  'interesting', 'better', 'differently',
'might', 'then', 'non', 'good', 'somebody',  'greater', 'downs', 'they', 'not',
'now', 'gets', 'always', 'l', 'each', 'went', 'side',  'everyone', 'year',
'our', 'out', 'opened', 'since', 'got', 'shows', 'turning', 'differ',  'quite',
'members', 'ask', 'wanted', 'g', 'could', 'needing', 'keep', 'thing', 'place',
'w', 'think', 'first', 'already', 'seeming', 'number', 'one', 'done',
'another', 'open',  'given', 'needed', 'ordering', 'least', 'anyone', 'their',
'too', 'gives', 'interests',  'mostly', 'behind', 'nobody', 'took', 'part',
'herself', 'than', 'kind', 'b', 'showed',  'older', 'likely', 'r', 'were',
'toward', 'and', 'sees', 'turned', 'few', 'say', 'have',  'need', 'seem',
'saw', 'orders', 'that', 'also', 'take', 'which', 'wanting', 'sure', 'shall',
'knew', 'wells', 'most', 'nothing', 'why', 'parting', 'noone', 'later', 'm',
'mrs', 'points', 'fact', 'show', 'ending', 'find', 'state', 'should', 'only',
'going', 'pointed', 'do', 'his', 'get', 'cannot', 'longest', 'during', 'him',
'areas', 'h', 'she', 'x', 'where', 'we', 'see', 'are', 'best', 'said', 'ways',
'away', 'enough', 'smallest',  'between', 'across', 'ends', 'never', 'opening',
'however', 'come', 'both', 'c', 'last',  'many', 'against', 's', 'became',
'faces', 'whole', 'asked', 'among', 'point', 'seems',  'furthered', 'furthers',
'puts', 'three', 'been', 'much', 'interest', 'wants', 'worked',  'an',
'present', 'case', 'myself', 'these', 'n', 'will', 'while', 'would', 'backing',
'is', 'thus', 'them', 'someone', 'in', 'if', 'different', 'perhaps', 'things',
'make',  'same', 'any', 'member', 'parts', 'several', 'higher', 'used', 'upon',
'uses', 'thoughts',  'off', 'largely', 'i', 'well', 'anybody', 'finds',
'thought', 'without', 'greatest',  'very', 'the', 'yours', 'latest', 'newest',
'just', 'less', 'being', 'when', 'rooms',  'facts', 'yet', 'had', 'lets',
'interested', 'has', 'gave', 'around', 'big', 'showing',  'possible', 'early',
'know', 'like', 'necessary', 'd', 't', 'fully', 'become', 'works',  'grouping',
'because', 'old', 'often', 'some', 'back', 'thinks', 'for', 'though', 'per',
'everything', 'does', 'either', 'be', 'who', 'seconds', 'nowhere', 'although',
'by', 'on',  'about', 'goods', 'asks', 'anything', 'of', 'o', 'or', 'into',
'within', 'down', 'beings',  'right', 'your', 'her', 'area', 'downed', 'there',
'long', 'way', 'was', 'opens', 'himself',  'but', 'newer', 'highest', 'with',
'he', 'made', 'places', 'whether', 'j', 'up', 'us',  'problem', 'z', 'clear',
'v', 'ordered', 'certain', 'general', 'as', 'at', 'face', 'again',  'no',
'generally', 'backs', 'grouped', 'other', 'you', 'really', 'felt', 'problems',
'important', 'sides', 'began', 'younger', 'e', 'longer', 'came', 'backed',
'together',  'u', 'presenting', 'evenly', 'having', 'once'])

In [ ]:
modernism_word_filt = modernism_word.mapValues(string.lower)\
                                    .mapValues(remove_punctuations)\
                                    .flatMapValues(string.split)\
                                    .filter(lambda pair: pair[1] not in stopwords)\
                                    .filter(lambda pair: len(pair[1]) > 3)\
                                    .filter(lambda pair: pair[1].isalpha())

In [ ]:
from operator import add
modernism_word_count = modernism_word_filt.values()\
                                          .map(lambda x: (x, 1))\
                                          .reduceByKey(add)

In [ ]:
modernism_word.count()

In [ ]:
modernism_word_count.top(10, key=lambda x: x[1])

### Advanced processing : Topic modelling

In [ ]:
# modern_top_vocab = list(map(lambda x: x[0], modernism_word_count.top(1000, key=lambda x: x[1])))
modern_top_vocab = modernism_word_count.keys().collect()
modern_vocab = dict(list(zip(modern_top_vocab, range(len(modern_top_vocab)))))
VEC_LENGTH = len(modern_vocab)

In [ ]:
br_modern_vocab = sc.broadcast(modern_vocab)

In [ ]:
from pyspark.mllib.linalg import Vectors
from operator import add
import numpy as np

def createCombiner(word):
    if word in br_modern_vocab.value:
        idx = br_modern_vocab.value[word]
        return Vectors.dense([0] * idx + [1] * (VEC_LENGTH - idx))
    else:
        return Vectors.dense([0] * VEC_LENGTH)

def mergeValue(vector, word):
    return vector + createCombiner(word)

In [ ]:
modernism_doc_word_count = modernism_word_filt.combineByKey(createCombiner, mergeValue, add)

In [ ]:
mdwc_idx = modernism_doc_word_count.map(lambda x: [hash(x[0]), x[1]])

In [ ]:
from pyspark.mllib.clustering import LDA

In [ ]:
numTopics = 10
ldaModel = LDA().train(mdwc_idx, k=numTopics, maxIterations=10)

In [ ]:
topicIndices = ldaModel.describeTopics(maxTermsPerTopic = 5)

In [ ]:
for terms, termWeights in topicIndices:
    print("TOPIC:")
    for term, weight in zip(terms, termWeights):
        print(modern_top_vocab[term], weight)
    print()

In [ ]:
sc.stop()